In [ ]:
#In this notebook we implement the methods used to compute the lovasz Theta number of a graph
# using the usual semidefinite formulation, and to compute the corresponding linear relaxation.
#relaxations of sparce pca. This notebook corresponds to section 4.3 
#of the paper

<h1>functions to compute the Lovasz Theta number SDP and its linear relaxation<h1>

<h2>Table of contents: <h2>   
    0. Functions, load packages.<br>

<h3>0. Functions, load packages.<br><h3>

In [ ]:
#Packages 
using JuMP,LinearAlgebra,MosekTools

In [2]:
#function to compute the lovasTheta number of a graph with adjacency matrix A. 
#We use the usual semidefinite formulation for this problem. 
#@ matrix A: adjancency matrix of the graph for which to compute the Theta number.
function lovazTheta(A)
n = size(A)[1]
    m = Model(Mosek.Optimizer)
    set_silent(m)
          @variable(m,X[1:n,1:n],PSD ) 
      @constraint(m,tr(X)==1)
      J = ones(n)*transpose(ones(n))
      for i in 1:n
        for j in 1:i 
            if A[i,j]== 1
    @constraint(m,X[i,j]==0)
    @constraint(m,X[j,i]==0)
            end
        end
    end
            
    @objective(m,Max,tr(J*X))
      status = optimize!(m)
    return(value.(X),objective_value(m))
end
 

lovazTheta (generic function with 1 method)

In [18]:
#function to compute the linear relaxation of the lovasz Theta number of a graph with adjacency matrix A. 
#@ matrix A: adjancency matrix of the graph for which to compute the linear relaxation.
function lovazThetaLP(A)

n = size(A)[1]
    m = Model(Mosek.Optimizer)
    set_silent(m)
          @variable(m,-1<=X[1:n,1:n]<=1,Symmetric ) 
      @constraint(m,tr(X)==1)
      J = ones(n)*transpose(ones(n))
    
     eigenFacto =  eigen(J-A)
    eigenVectors = eigenFacto.vectors
    eigenVectors = hcat(eigenVectors,eigen(A).vectors)
    eigenVectors = hcat(eigenVectors,Matrix{Float64}(I, n, n))
    eigenVectors = hcat(eigenVectors,eigen(J).vectors)
       for i in 1: size(eigenVectors)[2]
        v =  eigenVectors[:,i]
        @constraint(m,transpose(v)*X*v >=0)
    end
    for i in 1:n
        @constraint(m,X[i,i]>=0)
        for j in 1:i 
            if A[i,j]== 1
    @constraint(m,X[i,j]==0)
    @constraint(m,X[j,i]==0)
            end
        end
    end
    
 #    for i in 1:n
 #        for j in 1:n
 #           @constraint(m,[X[i,i]+X[j,j];2*X[i,j];X[i,i]-X[j,j]] in SecondOrderCone())
 #       end
 #   end
    
    @objective(m,Max,tr((J-A)*X))
      status = optimize!(m)
    return(value.(X),objective_value(m))
end


lovazThetaLP (generic function with 1 method)